In [1]:
from dotenv import load_dotenv
import os

import pandas as pd
from typing import Set
from transformers import GPT2TokenizerFast
import argparse, sys

import numpy as np

from PyPDF2 import PdfReader

import pandas as pd
import openai
import csv
import numpy as np
import os
import pickle
from transformers import GPT2TokenizerFast

In [6]:
# Use load_env to trace the path of .env:
load_dotenv('/home/alabarga/BSC/code/askmybook/.env')

True

In [7]:
openai.api_key = os.environ["OPENAI_API_KEY"]

In [8]:
COMPLETIONS_MODEL = "text-davinci-003"

MODEL_NAME = "curie"

DOC_EMBEDDINGS_MODEL = f"text-search-{MODEL_NAME}-doc-001"

In [9]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

In [14]:
def count_tokens(text: str) -> int:
    """count the number of tokens in a string"""
    return len(tokenizer.encode(text))

def extract_pages(
    page_text: str,
    index: int,
) -> str:
    """
    Extract the text from the page
    """
    if len(page_text) == 0:
        return []

    content = " ".join(page_text.split())
    print("page text: " + content)
    outputs = [("Page " + str(index), content, count_tokens(content)+4)]

    return outputs


In [15]:

def get_embedding(text: str, model: str) -> list[float]:
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    return result["data"][0]["embedding"]

def get_doc_embedding(text: str) -> list[float]:
    return get_embedding(text, DOC_EMBEDDINGS_MODEL)

def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str], list[float]]:
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.

    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_doc_embedding(r.content) for idx, r in df.iterrows()
    }


In [10]:
filename = '/home/alabarga/BSC/code/askmybook/books/TheBookOfOhdsi.pdf'

In [11]:
reader = PdfReader(filename)

In [12]:
from tqdm import tqdm

In [16]:
res = []
i = 1
for page in tqdm(reader.pages):
    res += extract_pages(page.extract_text(), i)
    i += 1


  1%|█▍                                                                                                                                         | 5/464 [00:00<00:11, 39.40it/s]

page text: The Book of OHDSI ObservationalHealthDataSciencesandInformatics 2020­11­25
page text: ii
page text: Contents Preface ix GoalsofthisBook . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ix StructureoftheBook . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ix Contributors . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . x SoftwareVersions . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . x License. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . xi HowtheBookIsDeveloped . . . . . . . . . . . . . . . . . . . . . . . . . . xi I The OHDSI Community 1 1 The OHDSI Community 3 1.1 TheJourneyfromDatatoEvidence . . . . . . . . . . . . . . . . . . . 3 1.2 ObservationalMedicalOutcomesPartnership . . . . . . . . . . . . . . 5 1.3 OHDSIasanOpen­ScienceCollaborative . . . . . . . . . . . . . . . . 6 1.4 OHDSI’sProgress . . . . . . . . . . . . . . . . . . . . . . . . . . . . 7 1.5 CollaboratinginOHDSI . . . . . . 

  6%|████████                                                                                                                                  | 27/464 [00:00<00:04, 87.99it/s]

page text: Chapter 1 The OHDSI Community Chapter leads: Patrick Ryan & George Hripcsak Coming together is a beginning; staying together is progress; working to­ getherissuccess. Henry Ford 1.1 The Journey from Data to Evidence Everywhereinhealthcare,allacrosstheworld,withinacademicmedicalcentersandpri­ vatepractices,regulatoryagenciesandmedicalproductmanufacturers,insurancecompa­ nies and policy centers, and at the heart of every patient­provider interaction, there is a common challenge: how do we apply what we’ve learned from the past to make better decisionsforthefuture? Formorethanadecade,manyhavearguedforthevisionofa learning healthcare sys­ tem, “designed to generate and apply the best evidence for the collaborative healthcare choices of each patient and provider; to drive the process of discovery as a natural out­ growthofpatientcare;andtoensureinnovation,quality,safety,andvalueinhealthcare”. (Olsenetal. ,2007)Achiefcomponentofthisambitionrestsontheexcitingprospectthat patient­le

 11%|██████████████▊                                                                                                                           | 50/464 [00:00<00:04, 98.35it/s]

...

page text: 384 Appendix A. Glossary populationsinorder tomimic randomizationbetween twotreatmentgroups inan observationalstudy. ThePSrepresentstheprobabilityofapatientreceivingatreat­ ment of interest as a function of a set of observed baseline covariates. It is most oftencalculatedusingalogisticregressionmodelwherethebinaryoutcomeisset to one for the group receiving the target treatment of interest and to zero for the comparatortreatment. SeeChapter 12. ProtocolAhumanreadabledocumentthatfullyspecifiesthedesignofastudy. Rabbit­in­a­Hat AninteractivesoftwaretooltohelpdefinetheETLfromsourceformat toCDM.UsesthedatabaseprofilegeneratedbyWhiteRabbitasinput. SeeChapter 7. Selection bias Abiasthatoccurswhenthesetofpatientsinyourdatadeviatesfromthe patientsinthepopulationinwaysthatdistortstatisticalanalyses. Self­controlled designs Study designs that compare outcomes during different expo­ sureswithinthesamepatient. Sensitivity analysis Avariantofthemainanalysisusedinastudytoassestheimpactof a

 95%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊      | 443/464 [00:04<00:00, 124.56it/s]

page text: E.5. SQL and R 415 Exercise 7.2 Probablynot. Defininganon­exposurecohortthatiscomparabletoyourdiclofenacexpo­ surecohortisoftenimpossible,sincepeopletakediclofenacforareason. Thisprecludesa between­personcomparison. Itmightpossibletoawithin­personcomparison,soforeach patientinthediclofenaccohortidentifyingtimewhentheyarenotexposed,butasimilar problemoccurshere: thesetimesarelikelyincomparable,becausetherearereasonswhen atonetimesomeoneisexposedandatothertimesnot. E.5 SQL and R Exercise 9.1 TocomputethenumberofpeoplewecansimplyquerythePERSONtable: library(DatabaseConnector) connection <- connect(connectionDetails) sql <-"SELECT COUNT(*) AS person_count FROM @cdm.person;" renderTranslateQuerySql (connection, sql, cdm ="main") ## PERSON_COUNT ## 1 2694 Exercise 9.2 Tocomputethenumberofpeoplewithatleastoneprescriptionofcelecoxib,wecanquery theDRUG_EXPOSUREtable. Tofindalldrugscontainingtheingredientcelecoxib,we jointotheCONCEPT_ANCESTORandCONCEPTtables: library(DatabaseConnector

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 464/464 [00:05<00:00, 92.12it/s]

page text: Bibliography 443 ACE inhibitor­induced angioedema after eleven years on lisinopril. J Pharm Pract , 26(4):382–388. Oliveira, J.L., Trifan, A., andSilva, L.A.B.(2019). EMIFcatalogue: Acollaborative platform for sharing and reusing biomedical data. International Journal of Medical Informatics ,126:35–45. Olsen, L., Aisner, D., McGinnis, J. M., et al. (2007). The learning healthcare system: workshop summary . NatlAcademyPr. O’Mara,N.B.andO’Mara,E.M.(1996).Delayedonsetofangioedemawithangiotensin­ convertingenzymeinhibitors: casereportandreviewoftheliterature. Pharmacother­ apy,16(4):675–679. Overhage, J. M., Ryan, P. B., Reich, C. G., Hartzema, A. G., and Stang, P. E. (2012). Validationofacommondatamodelforactivesafetysurveillanceresearch. J Am Med Inform Assoc ,19(1):54–60. Perkins,N.J.,Cole,S.R.,Harel,O.,TchetgenTchetgen,E.J.,Sun,B.,Mitchell,E.M., andSchisterman,E.F.(2017). Principledapproachestomissingdatainepidemiologic studies. American journal of epidemiology ,187(3):568–5

In [17]:
df = pd.DataFrame(res, columns=["title", "content", "tokens"])
df = df[df.tokens<2046]
df = df.reset_index().drop('index',axis=1) # reset index
df.head()

,title,content,tokens
0,Page 1,The Book of OHDSI ObservationalHealthDataScien...,26
1,Page 2,ii,5
2,Page 3,Contents Preface ix GoalsofthisBook . . . . . ...,807
3,Page 4,iv Contents II Uniform Data Representation 29 ...,1136
4,Page 5,Contents v 8.1 AnalysisImplementation . . . . ...,1307


In [18]:
from pathlib import Path

In [26]:
dir(Path(filename))

['__bytes__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__fspath__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rtruediv__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__truediv__',
 '_accessor',
 '_cached_cparts',
 '_cparts',
 '_drv',
 '_flavour',
 '_format_parsed_parts',
 '_from_parsed_parts',
 '_from_parts',
 '_hash',
 '_make_child',
 '_make_child_relpath',
 '_parse_args',
 '_parts',
 '_pparts',
 '_root',
 '_str',
 'absolute',
 'anchor',
 'as_posix',
 'as_uri',
 'chmod',
 'cwd',
 'drive',
 'exists',
 'expanduser',
 'glob',
 'group',
 'hardlink_to',
 'home',
 'is_absolute',
 'is_block_device',
 'is_char_device',
 'is_dir',
 'is_fifo',
 'is_file',
 'is_mount',
 'is_relative_to',
 'is_reserved',
 'is_socket

In [30]:
help(Path(filename).rename)


Help on method rename in module pathlib:

rename(target) method of pathlib.PosixPath instance
    Rename this path to the target path.
    
    The target path may be absolute or relative. Relative paths are
    interpreted relative to the current working directory, *not* the
    directory of the Path object.
    
    Returns the new Path instance pointing to the target path.



In [35]:
pages_file = Path(filename).with_suffix('.pages.csv')
embeddings_file = Path(filename).with_suffix('.embeddings.csv')

In [33]:
df.to_csv(pages_file, index=False)

In [36]:
# CSV with exactly these named columns:
# "title", "0", "1", ... up to the length of the embedding vectors.

doc_embeddings = compute_doc_embeddings(df)

RateLimitError: You exceeded your current quota, please check your plan and billing details.

In [ ]:
with embeddings_file.open('w') as f:
    writer = csv.writer(f)
    writer.writerow(["title"] + list(range(4096)))
    for i, embedding in list(doc_embeddings.items()):
        writer.writerow(["Page " + str(i + 1)] + embedding)